In [13]:
import pandas as pd
import numpy as np
from scipy import stats

rs_contract = pd.read_excel("../../Datasets/study-1-data.xlsx", sheet_name="RS contract")

pd.set_option('display.max_rows', 100)

In [14]:
rs_contract = rs_contract.drop("Unnamed: 6", axis=1)

In [15]:
rs_contract

,Experiment,Period,Player ID,Demand,Wholesale p.,Revenue Share,Player ID.1,Stock
0,1,1,Manufacturer 1,122,3,5,Retailer 1,90
1,1,2,Manufacturer 1,144,0,8,Retailer 1,150
2,1,3,Manufacturer 1,57,2,7,Retailer 1,0
3,1,4,Manufacturer 1,149,2,6,Retailer 1,100
4,1,5,Manufacturer 1,135,3,4,Retailer 1,110
...,...,...,...,...,...,...,...,...
875,4,36,Manufacturer 22,136,3,4,Retailer 22,82
876,4,37,Manufacturer 22,116,4,3,Retailer 22,74
877,4,38,Manufacturer 22,105,5,2,Retailer 22,103
878,4,39,Manufacturer 22,80,5,2,Retailer 22,106


In [16]:
rs_contract.columns

Index(['Experiment', 'Period', 'Player ID', 'Demand', 'Wholesale p.',
       'Revenue Share', 'Player ID.1', 'Stock'],
      dtype='object')

In [18]:
rs_contract['Sales'] = rs_contract[['Demand', 'Stock']].min(axis=1)
rs_contract['Leftovers'] = rs_contract['Stock'] - rs_contract['Sales']
rs_contract['Unmet_Demand'] = rs_contract['Demand'] - rs_contract['Sales']



rs_contract['Realized_Retailer_Profit'] = rs_contract['Sales'] * 12 - (rs_contract['Stock'] * rs_contract['Wholesale p.']) - rs_contract["Revenue Share"] * rs_contract["Sales"]
rs_contract['Realized_Mfg_Profit'] = (rs_contract['Stock'] * rs_contract['Wholesale p.']) - (3 * rs_contract['Stock']) + rs_contract["Revenue Share"] * rs_contract["Sales"]

rs_contract['Realized_Mfg_Profit_Share'] = rs_contract['Realized_Mfg_Profit'] / (rs_contract['Realized_Mfg_Profit'] + rs_contract['Realized_Retailer_Profit'])

#wsp_contract['Optimal_Stock'] = 100*((12-wsp_contract['Wholesale p.'])/12 - 'Buyback p.') + 50
rs_contract['Optimal_Stock'] =  100 * (      (12 - rs_contract['Wholesale p.'] - rs_contract["Revenue Share"]) /  (   12 - rs_contract['Revenue Share']    )    )       +    50
rs_contract['Optimal_Stock'] = rs_contract['Optimal_Stock'].round()

rs_contract['Under/Overstock_wrt_Optimal'] = rs_contract['Stock'] - rs_contract['Optimal_Stock']
potential_demand = np.arange(51, 151)
expected_sales = []

for index, row in rs_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

rs_contract['Expected Sales'] = np.array(expected_sales)
rs_contract["Expected_Leftovers"] = rs_contract["Optimal_Stock"] - rs_contract["Expected Sales"]

#wsp_contract['Expected Retailer Profit'] = (wsp_contract['Expected Sales']  12) - (wsp_contract['Wholesale p.'] * wsp_contract['Stock']) + wsp_contract["Expected_Leftovers"] * ..buyback price..
rs_contract['Expected Retailer Profit'] = (rs_contract['Expected Sales'] * 12) - (rs_contract['Wholesale p.'] * rs_contract['Stock']) - rs_contract["Revenue Share"] * rs_contract["Expected Sales"]

#wsp_contract['Expected Mfg Profit'] = (wsp_contract['Stock']  wsp_contract['Wholesale p.']) - (3  wsp_contract['Stock']) - ..buybackprice.. * wsp_contract["Expected_Leftovers"]
rs_contract['Expected Mfg Profit'] = (rs_contract['Stock'] * rs_contract['Wholesale p.']) - (3 * rs_contract['Stock']) + rs_contract["Revenue Share"] * rs_contract["Expected Sales"]

rs_contract['Expected Mfg. Profit Share'] = rs_contract['Expected Mfg Profit'] / (rs_contract['Expected Mfg Profit'] + rs_contract['Expected Retailer Profit']) #rs_contract.apply(lambda row: row['Expected Mfg Profit'] / row['Total Expected Revenue'] if row['Total Expected Revenue'] != 0 else 0, axis=1)
potential_demand = np.arange(51, 151)
expected_sales = []

for index, row in rs_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Optimal_Stock'])

        cumulative_expected_sales += sales
        
        #if sales == row['Stock']:
         #   cumulative_expected_sales += (150 - demand_realization) * row['Stock']
          #  break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

rs_contract['Predicted Sales'] = np.array(expected_sales)
rs_contract['Predicted Leftovers'] = rs_contract['Optimal_Stock'] - rs_contract['Predicted Sales'] 

rs_contract['Predicted Retailer Profit'] = rs_contract['Predicted Sales'] * 12 - (rs_contract['Wholesale p.'] * rs_contract['Optimal_Stock']) - rs_contract["Revenue Share"] * rs_contract["Predicted Sales"]

rs_contract['Predicted Mfg Profit'] = (rs_contract['Optimal_Stock'] * rs_contract['Wholesale p.']) - (3 * rs_contract['Optimal_Stock']) + rs_contract["Revenue Share"] * rs_contract["Predicted Sales"]
rs_contract['Predicted Mfg. Profit Share'] = rs_contract['Predicted Mfg Profit'] / (rs_contract['Predicted Mfg Profit'] + rs_contract['Predicted Retailer Profit'])

In [19]:
rs_contract.to_csv('./curr_data_rs.csv', index = False)